In [21]:
#!pip install -U hopsworks --quiet

In [78]:
import random
import pandas as pd
import hopsworks

BACKFILL=False

In [79]:
def generate_magnetic_field_update(name, x_max, x_min, y_max, y_min, z_max, z_min):
    """
    Returns a single magnetic field update as a single row in a DataFrame
    """
    df = pd.DataFrame({ "x": [random.uniform(x_max, x_min)],
                       "y": [random.uniform(y_max, y_min)],
                       "z": [random.uniform(z_max, z_min)],
                      })
    df['mag_cluster'] = name
    return df


def get_random_magnetic_field_update():
    """
    Returns a DataFrame containing one random magnetic field update
    """
    detection_df = generate_magnetic_field_update("detection", 1156, -374, 401, -1275, 603, -1433)
    no_detection_df = generate_magnetic_field_update("no_detection", 2041, -860, 905, -897, -133, -4087)

    # randomly pick one of these 2 and write it to the featurestore
    pick_random = random.uniform(0,2)
    if pick_random >= 1:
        mag_df = detection_df
    else:
        mag_df = no_detection_df

    return mag_df

In [68]:
df_main =pd.read_csv("/workspaces/MLOps_Project/data/EL123.csv")

In [69]:
df_main = df_main[['x','y','z','mag_cluster']]

In [70]:
df_main['mag_cluster'].replace({0: 'detection', 1: 'no_detection'}, inplace=True)

In [72]:
df_main[['x','y','z']] = df_main[['x','y','z']].astype(float)

In [74]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20570 entries, 0 to 20569
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   x            20570 non-null  float64
 1   y            20570 non-null  float64
 2   z            20570 non-null  float64
 3   mag_cluster  20570 non-null  object 
dtypes: float64(3), object(1)
memory usage: 642.9+ KB


In [89]:
if BACKFILL == True:
    mag_df = df_main
else:
    mag_df = get_random_magnetic_field_update()
    
mag_df.head()

,x,y,z,mag_cluster
0,-241.09827,-549.593766,-858.673455,detection


In [81]:
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549014
Connected. Call `.close()` to terminate connection gracefully.


In [90]:
mag_fg = fs.get_or_create_feature_group(name="mag_feature",
                                  version=1,
                                  primary_key=["x","y","z"],
                                  description="Magnetic field dataset"
                                 )
mag_fg.insert(mag_df)

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: mag_feature_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549014/jobs/named/mag_feature_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f548c208730>, None)